In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Define the transforms for the dataset
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the train and test datasets

train_dataset = datasets.Flowers102(root="./data", split="train", transform=transform, download=True)
test_dataset = datasets.Flowers102(root="./data", split="test", transform=transform, download=True)

# Define the CNN architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(28 * 28 * 256, 1024)
        self.fc2 = nn.Linear(1024, 102)
        
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 28 * 28 * 256)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create an instance of the CNN and move it to the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cnn = CNN().to(device)

# Define the loss function and the optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters())

# Train the CNN
batch_size = 64
num_epochs = 20
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
for epoch in range(num_epochs):
    cnn.train() # set the model in train mode
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if (i+1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}")
            
    # Evaluate the model on the test set after every epoch
    cnn.eval() # set the model in evaluation mode
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = cnn(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        test_accuracy = 100 * correct / total
        print(f"Test Accuracy after epoch {epoch+1}: {test_accuracy:.2f}%")

# Final test accuracy
print
(f"Test Accuracy: {(100 * correct / total):.2f}%")


Epoch [1/20], Step [10/16], Loss: 4.6145
Test Accuracy after epoch 1: 2.33%
Epoch [2/20], Step [10/16], Loss: 4.3533
Test Accuracy after epoch 2: 4.46%
Epoch [3/20], Step [10/16], Loss: 3.6351
Test Accuracy after epoch 3: 6.34%
Epoch [4/20], Step [10/16], Loss: 2.8710
Test Accuracy after epoch 4: 11.89%
Epoch [5/20], Step [10/16], Loss: 1.8569
Test Accuracy after epoch 5: 14.85%
Epoch [6/20], Step [10/16], Loss: 1.1881
Test Accuracy after epoch 6: 15.24%
Epoch [7/20], Step [10/16], Loss: 0.5251
Test Accuracy after epoch 7: 13.68%
Epoch [8/20], Step [10/16], Loss: 0.2574
Test Accuracy after epoch 8: 13.92%
Epoch [9/20], Step [10/16], Loss: 0.4507
Test Accuracy after epoch 9: 13.11%
Epoch [10/20], Step [10/16], Loss: 0.1330
Test Accuracy after epoch 10: 12.05%
Epoch [11/20], Step [10/16], Loss: 0.2159
Test Accuracy after epoch 11: 15.40%
Epoch [12/20], Step [10/16], Loss: 0.0046
Test Accuracy after epoch 12: 14.51%
Epoch [13/20], Step [10/16], Loss: 0.0044
Test Accuracy after epoch 13: 1

'Test Accuracy: 14.12%'